# Running Nanopyx on data from OMERO

This notebook allows you to perform channel registration of 2D+t multi-channel image stacks using data on OMERO.

If you use this notebook found this work useful for your research please cite [preprint](https://www.biorxiv.org/content/10.1101/2023.08.13.553080v1), the Nanopyx Github repo [![DOI](https://zenodo.org/badge/505388398.svg)](https://zenodo.org/badge/latestdoi/505388398) and [acknowledge CAMDU](https://warwick.ac.uk/fac/sci/med/research/biomedical/facilities/camdu/acknowledgementpolicy/)

**Channel Registration** Romain F Laine et al 2019 J. Phys. D: Appl. Phys. 52 163001




In [2]:
import io
import os
import cv2 as cv
import skimage
import nanopyx
import stackview
import numpy as np
import tifffile as tiff
import matplotlib as mpl
import ipywidgets as widgets
from PIL import Image
from IPython.display import display, clear_output
from matplotlib import pyplot as plt
from omero.gateway import BlitzGateway
import getpass

from nanopyx.core.utils.easy_gui import EasyGui
from nanopyx.core.utils.find_files import find_files
from nanopyx.data.download import ExampleDataManager

from nanopyx.core.generate.beads import generate_channel_misalignment

Enter OMERO login details:

In [ ]:
username = input('Username: ') #Request user to enter OMERO username
password = getpass.getpass(prompt='Password: ') #Request user to enter OMERO password

Load image to use to calculate channel registation. Enter the image ID from OMERO.

In [32]:
global dataset_original

def get_stack(img):
    """
    Convert OMERO image object to numpy array
    Input: img  OMERO image object
    """
    zct_list = [(z, c, t) for z in range(img.getSizeZ()) for c in range(img.getSizeC()) for T in range(img.getSizeT())] #Set dimensions of image
    pixels = img.getPrimaryPixels()
    return np.array(list(pixels.getPlanes(zct_list))) #Read in data one plane at a time

image_id = input('Image ID: ')
conn = BlitzGateway(username, password, host='white-pike.lnx.warwick.ac.uk', port=4064)
conn.connect() #Returns True when connected
image = conn.getObject("Image", image_id)
name = image.getName()
print("Loading...")
dataset_original=get_stack(image)
z = image.getSizeZ() / 2 #Middle z plane
t = 0 #First time point
rendered_image=image.renderImage(z,t) #Rendering settings from OMERO.web
rendered_image.show() #Opens pop up
conn.close()
dataset_original.shape


Image ID:  809560


Loading...


(4, 1196, 1196)

# Channel Registration Parameters:  

Requires an image stack with shape: (channel, rows, columns).  

- **Reference Channel:** Which channel to be used as reference.  
- **Max Expected Shift:** Maximum amount of expected shift between channels, in pixels.  
- **Blocks per Axis:** As channel misalignmnet is not always homogeneous across the field of view, shift can be calculated for individual blocks of the field of view. This parameters sets how many blocks are created along both axis.  
- **Minimum Similarity:** Since smaller blocks may lead to shift calculation in areas of the image without any cells, minimum similarity can be used to define the minimum Pearson's Correlation Coefficient, between two blocks of different channels, required to use the calculated shifts as part of the registration. 




In [34]:
#@title Create channel registration GUI
from nanopyx.methods import channel_registration

gui_reg = EasyGui("Channel Registration")

def on_button_register(b):
    clear_output()
    gui_reg.show()
    ref_channel = gui_reg["ref"].value
    max_shift = gui_reg["max"].value
    n_blocks = gui_reg["blocks"].value
    min_sim = gui_reg["min_sim"].value
    global dataset_registered
    gui_reg["register"].disabled = True
    gui_reg["register"].description = "Aligning..."
    if gui_reg["save"].value:
        save_translation_masks = True
        if name:
            save_path = name + "_registered.tif"
            translation_mask_path = name
        else:
            disp("No image found")
    else:
        save_translation_masks = False
        translation_mask_path = ""
    dataset_registered = channel_registration.estimate_channel_registration(dataset_original,
                                                                            ref_channel,
                                                                            max_shift,
                                                                            n_blocks,
                                                                            min_sim,
                                                                            save_translation_masks=save_translation_masks,
                                                                            translation_mask_save_path=translation_mask_path,
                                                                            apply=True)
    if gui_reg["save"].value:
        tiff.imwrite(save_path, dataset_registered)
    gui_reg["register"].disabled = False
    gui_reg["register"].description = "Align"
    gui_reg._main_display.children = gui_reg._main_display.children + (stackview.slice(dataset_registered, colormap=gui_reg["cmaps"].value, continuous_update=True),)


gui_reg.add_label("Channel Registration parameters:")
gui_reg.add_int_slider("ref", description="Reference channel", min=0, max=dataset_original.shape[0]-1, value=0)
gui_reg.add_int_slider("max", description="Max expected drift", min=0, max=1000, value=10)
gui_reg.add_int_slider("blocks", description="Blocks per axis", min=1, max=10, value=5)
gui_reg.add_float_slider("min_sim", description="Minimum similarity", min=0, max=1, value=0.5, step=0.1)
gui_reg.add_dropdown("cmaps", description="Colormap:",
                     options=sorted(list(mpl.colormaps)),
                     value="viridis", remember_value=True)
gui_reg.add_checkbox("save", description="Save Output", value=True)
gui_reg.add_button("register", description="Register")
gui_reg["register"].on_click(on_button_register)
gui_reg.show()




## Use the following cells only if you have a previously calculated translation mask



Enter ID of Dataset from OMERO

In [15]:
dataset_id = input('Dataset ID: ')

Dataset ID:  41364


In [40]:
#@title Create channel registration GUI
import sys
from nanopyx.methods.channel_registration import apply_channel_registration
from IPython.display import display, clear_output

def get_t_stack(img,t):
    """
    Convert OMERO image object to numpy array one time point at a time
    Input: img  OMERO image object
    """
    zct_list = [(z, c, t) for z in range(img.getSizeZ()) for c in range(img.getSizeC())] #Set dimensions of image
    pixels = img.getPrimaryPixels()
    return np.array(list(pixels.getPlanes(zct_list))) #Read in data one plane at a time

gui_reg_apply = EasyGui("Channel Registration Apply")

def on_button_apply(b):
    clear_output()
    gui_reg_apply.show()

    gui_reg_apply["Register Image"].disabled = True
    #gui_reg_apply["Register Image"].description = "Aligning..."

    translation_mask_path = gui_reg_apply["upload"].selected
    translation_mask = tiff.imread(translation_mask_path)
    conn = BlitzGateway(username, password, host='white-pike.lnx.warwick.ac.uk', port=4064)
    conn.connect() #Returns True when connected
    dataset = conn.getObject("Dataset", dataset_id)
    for img in dataset.listChildren():
        name = img.getName()
        final_stack = []
        gui_reg_apply["Register Image"].description = "Aligning " + name + "..."
        for t in range(img.getSizeT()):
            image_to_align = get_t_stack(img,t)
            aligned_image = apply_channel_registration(image_to_align, translation_mask)
            final_stack.append(aligned_image)
    
        if gui_reg_apply["save"].value:
            save_path = name + "_registered.tif"
            tiff.imwrite(save_path, final_stack)

    conn.close()

    gui_reg_apply["Register Image"].disabled = False
    gui_reg_apply["Register Image"].description = "Align"
    gui_reg_apply._main_display.children = gui_reg_apply._main_display.children + (stackview.slice(aligned_image, colormap=gui_reg_apply["cmaps"].value, continuous_update=True),)

gui_reg_apply.add_label("Load translation mask:")
gui_reg_apply.add_file_upload("upload")
gui_reg_apply.add_dropdown("cmaps", description="Colormap:",
                      options=sorted(list(mpl.colormaps)),
                      value="viridis", remember_value=True)
gui_reg_apply.add_checkbox("save", description="Save Output", value=True)
gui_reg_apply.add_button("Register Image", description="Register Image")
gui_reg_apply["Register Image"].on_click(on_button_apply)
gui_reg_apply.show()

